In [5]:
import json, re, gzip
import requests
from urlextract import URLExtract

#for this project, the only input files you will see are my own
#every other input file was deleted

utid = 'yarddoga'
base_urls = {'model': 'https://huggingface.co/', 'data': 'https://huggingface.co/datasets/', 'source': 'https://'}
post = '/raw/main/README.md'
postGH = 'blob/master/README.md'

extU = URLExtract()
DOIpattern = r'\b(10\.\d{4,9}\/[-._;()/:A-Z0-9]+)\b/i'
#r1\b(10[.][0-9]{4,}(?:[.][0-9]+)*/(?:(?!["&\'<>])[[:graph:]])+)\b'

def extractURLs(c):
    return extU.find_urls(c)

def extractDOIs(c):
    return re.findall(DOIpattern, c)

def process_type(tp):
    with open(f"input/{utid}_{tp}", 'r') as f:
        for line in f:
            line = line.strip()
            post_suffix = postGH if tp == 'source' else post
            id = line.split(';')[1] if tp == 'source' else line
            
            url = f"{base_urls[tp]}{id}{post_suffix}"
            r = requests.get(url)
            if r.status_code != 200:
                continue
            
            content = r.text.replace('\n', ' ')
            urls = extractURLs(content)
            dois = extractDOIs(content)
            
            res = {'id': id, 'type': tp, 'url': url, 'content': content,'links': urls, 'dois': dois, 'bibs': []}
            yield res

with gzip.open(f"output/{utid}.json.gz", 'wt', encoding='utf-8') as fo:
    for tp in ['model', 'data', 'source']:
        for entry in process_type(tp):
            json.dump(entry, fo)
            fo.write('\n')